In [2]:
# from typing import override

In [3]:
import os
from typing import override

import numpy as np
np.random.seed(2025-7-23)

import torch
import torch.nn as nn
from torch.nn.functional import pad

In [7]:
# just pick 100 images of cars (nonrepetitively)

for image in np.random.choice(os.listdir(r"D:/carvana/"), size=100, replace=False):
    os.rename(src=f"D:/carvana/{")

array(['8a34d46ec604_16.jpg', '74438e264e85_03.jpg',
       '0dfde35f5c87_04.jpg', 'd9df232dc92c_10.jpg',
       '46562684485a_16.jpg', '21c783b9841b_15.jpg',
       'ae83ca58bb60_08.jpg', 'df74648c7553_11.jpg',
       'b929b3a41d2e_15.jpg', 'b66c26e01b3f_07.jpg',
       '292ce2c1c76b_13.jpg', 'fe0acb1321be_09.jpg',
       'e8c01181f5e6_12.jpg', 'e9113fe5e69d_12.jpg',
       '047df42a3f14_05.jpg', '16e8a5e36d36_16.jpg',
       '87a4a64b3632_14.jpg', '9e139de2da5d_08.jpg',
       '94872c4309c8_04.jpg', '3bfff4279409_05.jpg',
       'bc2bd280d9b8_16.jpg', '1556fb3462b8_09.jpg',
       'ae3ee849ed63_03.jpg', '3b69d6995ace_08.jpg',
       '754eed53c22b_07.jpg', 'c6dfca36ecfc_01.jpg',
       '90bdde690e23_08.jpg', '5b25e7d531b2_01.jpg',
       'd465eff90a85_16.jpg', 'c7f510bbfb1d_09.jpg',
       'ff190ce147ac_07.jpg', 'a9886f0dda80_10.jpg',
       '7e097d195756_14.jpg', 'a715500c187f_04.jpg',
       '9da7e05d7982_02.jpg', 'bbecfb9145d8_03.jpg',
       'e32c4ab15d7f_08.jpg', 'af89f7cd4e98_03

# ___UNet___
---------------------

In [ ]:
class DoubleConv(nn.Module):
    """
    (convolution => [BN] => ReLU) * 2
    """

    def __init__(self, in_channels: int, out_channels: int, mid_channels: int | None = None) -> None:
        super(DoubleConv, self).__init__()  # type: ignore

        if not mid_channels:
            mid_channels = out_channels

        self.double_conv = nn.Sequential(
            # first set
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            # second set
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    @override
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.double_conv(x)


class Down(nn.Module):
    """
    Downscaling with maxpool then double convolution
    """

    def __init__(self, in_channels: int, out_channels: int) -> None:
        super(Down, self).__init__()  # type: ignore
        self.maxpool_conv = nn.Sequential(nn.MaxPool2d(2), DoubleConv(in_channels, out_channels))

    @override
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.maxpool_conv(x)


class Up(nn.Module):
    """
    Upscaling then double convolution
    """

    def __init__(self, in_channels: int, out_channels: int, bilinear: bool = True) -> None:
        super(Up, self).__init__()  # type: ignore

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    @override
    def forward(self, x1: torch.Tensor, x2: torch.Tensor) -> torch.Tensor:
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = pad(x1, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels: int, out_channels: int) -> None:
        super(OutConv, self).__init__()  # type: ignore
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    @override
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.conv(x)


class UNet(nn.Module):
    """ """

    def __init__(self, n_channels: int, n_classes: int, bilinear: bool = False) -> None:
        super(UNet, self).__init__()  # type: ignore
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        factor: int = 2 if bilinear else 1

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)

        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    @override
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

    def use_checkpointing(self) -> None:
        self.inc = torch.utils.checkpoint(self.inc)
        self.down1 = torch.utils.checkpoint(self.down1)
        self.down2 = torch.utils.checkpoint(self.down2)
        self.down3 = torch.utils.checkpoint(self.down3)
        self.down4 = torch.utils.checkpoint(self.down4)
        self.up1 = torch.utils.checkpoint(self.up1)
        self.up2 = torch.utils.checkpoint(self.up2)
        self.up3 = torch.utils.checkpoint(self.up3)
        self.up4 = torch.utils.checkpoint(self.up4)
        self.outc = torch.utils.checkpoint(self.outc)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(n_channels=3, n_classes=1)
model.to(device=device)